In [1]:
!pip install pycocotools

  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=272452 sha256=822a0c3ee6ed1b8e7b741378506d1fbcc5610a4330d7fed4ec3be5172bd1ffcd
  Stored in directory: /root/.cache/pip/wheels/bc/cf/1b/e95c99c5f9d1648be3f500ca55e7ce55f24818b0f48336adaf
Successfully built pycocotools


In [2]:
!cp ../input/torchdetection/* .

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd 
import math

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import sys

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#my imports
import pydicom as dicom
import pickle
import matplotlib as plt
import matplotlib.patches as patches
import pandas as pd
import numpy as np
import PIL
from PIL import Image

import torch
import transforms as T
import torchvision.transforms as T2
import utils
import torchvision
import tqdm


import warnings
warnings.filterwarnings("ignore")

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [5]:
csv = pd.read_csv("../input/vinbigdata-1024-image-dataset/vinbigdata/train.csv")

In [6]:
def custom_normalize(img, mean, std, max_pixel_value=255.0):
    mean = np.array(mean, dtype=np.float32)
    mean *= max_pixel_value

    std = np.array(std, dtype=np.float32)
    std *= max_pixel_value

    denominator = np.reciprocal(std, dtype=np.float32)

    img = img.astype(np.float32)
    img -= mean
    img *= denominator
    return img

In [7]:
class ChestXRayDataset(object):
    def __init__ (self, path, tfms, csv, weights=None):
        #defaults
        self.path=path
        self.tfms=tfms
        self.csv = csv
        self.weights = weights
        #images
        self.imgs = [x+".png" for x in csv['image_id'].tolist()]
    def __getitem__ (self, idx):
        img_path = os.path.join(self.path, 'train', self.imgs[idx])
        img = Image.open(img_path)
        # img = np.asarray(img)
        # img/np.max(img)
        
        boxes = []
        labs = []
        ic = []
        dk = self.csv.loc[self.csv['image_id']==self.imgs[idx][:-4]]
        area = []
        
        for idx, row in dk.iterrows():
            if (int(row['class_id'])!=14): 
                labs.append(int(row['class_id'])+1)
                ic.append(0)
                xmin = float(row['x_min'])
                xmax = float(row['x_max'])
                ymin = float(row['y_min'])
                ymax = float(row['y_max'])

                xmin*=1024
                xmin/=row['width']
                xmax*=1024
                xmax/=row['width']
                ymin*=1024
                ymin/=row['height']
                ymax*=1024
                ymax/=row['height']
                # if (img.shape[0]>img.shape[1]):
                #     rat = 800/img.shape[0]
                #     xmin = rat*xmin
                #     xmax = rat*xmax
                #     ymin = rat*ymin
                #     ymax = rat*ymax
                # else:
                #     rat = 800/img.shape[1]
                #     xmin = rat*xmin
                #     xmax = rat*xmax
                #     ymin = rat*ymin
                #     ymax = rat*ymax
                # xmin = floor(xmin)
                # ymin = floor(ymin)
                # xmax = floor(xmax)
                # ymax = floor(ymax)
                boxes.append([xmin, ymin, xmax, ymax])
                area.append((xmax-xmin)*(ymax-ymin))
                
        if (len(boxes)==0):
            boxes = [[0, 0, 1, 1]]
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.zeros((1,), dtype=torch.int64)
            area = torch.ones((1,), dtype=torch.float32)
            iscrowd = torch.as_tensor([0],dtype=torch.int64)
        else:
            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            labels = torch.as_tensor(labs, dtype=torch.int64)
            area = torch.as_tensor(area, dtype=torch.float32)
            iscrowd = torch.as_tensor(ic, dtype=torch.int64)
            
        image_id = torch.tensor([idx])
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd
        
        if self.tfms is not None:
#             img = custom_normalize(img, (0,0,0), (1,1,1))
            # img = Image.fromarray(img)
            img = self.tfms(img)
        return img, target
    def __len__(self):
        return len(self.imgs)

In [8]:
def get_transform():
    transform = []
    transform.append(T2.ToTensor())
    
    return T2.Compose(transform)

In [9]:
file_train = open(f"../input/private-model-save-dataset/train-set-v3 (1).pkl", "rb")
ds_train = pickle.load(file_train)

In [10]:
ds_train.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,2332,2580
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0,2080,2336
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN,2540,3072
5,1c32170b4af4ce1a3030eb8167753b06,Pleural thickening,11,R9,627.0,357.0,947.0,433.0,2540,3072
6,0c7a38f293d5f5e4846aa4ca6db4daf1,ILD,5,R17,1347.0,245.0,2188.0,2169.0,2285,2555


In [11]:
csv.head()

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height
0,50a418190bc3fb1ef1633bf9678929b3,No finding,14,R11,NaN,NaN,NaN,NaN,2332,2580
1,21a10246a5ec7af151081d0cd6d65dc9,No finding,14,R7,NaN,NaN,NaN,NaN,2954,3159
2,9a5094b2563a1ef3ff50dc5c7ff71345,Cardiomegaly,3,R10,691.0,1375.0,1653.0,1831.0,2080,2336
3,051132a778e61a86eb147c7c6f564dfe,Aortic enlargement,0,R10,1264.0,743.0,1611.0,1019.0,2304,2880
4,063319de25ce7edb9b1c6b8881290140,No finding,14,R10,NaN,NaN,NaN,NaN,2540,3072


In [12]:
ds_test = pd.merge(csv,ds_train, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)

In [13]:
ds_test[ds_test['image_id']=='afb6230703512afc370f236e8fe98806']

,image_id,class_name,class_id,rad_id,x_min,y_min,x_max,y_max,width,height
9,afb6230703512afc370f236e8fe98806,Pulmonary fibrosis,13,R9,1857.0,1607.0,2126.0,2036.0,2540,3072
2984,afb6230703512afc370f236e8fe98806,Lung Opacity,7,R9,486.0,1594.0,1002.0,1918.0,2540,3072
5196,afb6230703512afc370f236e8fe98806,ILD,5,R8,494.0,1385.0,1148.0,1919.0,2540,3072
14545,afb6230703512afc370f236e8fe98806,Aortic enlargement,0,R10,1331.0,762.0,1653.0,1081.0,2540,3072
15534,afb6230703512afc370f236e8fe98806,ILD,5,R10,488.0,1602.0,1093.0,1931.0,2540,3072
24425,afb6230703512afc370f236e8fe98806,Pulmonary fibrosis,13,R10,1870.0,1591.0,2107.0,1990.0,2540,3072
30045,afb6230703512afc370f236e8fe98806,Pulmonary fibrosis,13,R9,486.0,1594.0,1002.0,1918.0,2540,3072
31422,afb6230703512afc370f236e8fe98806,Lung Opacity,7,R9,1857.0,1607.0,2126.0,2036.0,2540,3072
34077,afb6230703512afc370f236e8fe98806,ILD,5,R10,1870.0,1591.0,2107.0,1990.0,2540,3072
35170,afb6230703512afc370f236e8fe98806,Aortic enlargement,0,R8,1352.0,742.0,1648.0,1129.0,2540,3072


In [14]:
test_imgs = ds_test['image_id'].unique()
test_imgs

array(['21a10246a5ec7af151081d0cd6d65dc9',
       '051132a778e61a86eb147c7c6f564dfe',
       'd3637a1935a905b3c326af31389cb846', ...,
       '913386914d3d68400a45e88f7f2e8d4d',
       '209e3a5c73c1195d805dd25d086b3c6d',
       '1bea772246421c37929f8cbc43dba729'], dtype=object)

In [15]:
def bb_intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [16]:
for i in test_imgs:
    df = ds_test.loc[ds_test['image_id']==i]
    i = 0
    while (i<len(df)):
        j = i+1
        while (j<len(df)):
            if bb_intersection_over_union([df.iloc[i]['x_min'], df.iloc[i]['y_min'], df.iloc[i]['x_max'], df.iloc[i]['y_max']], 
                    [df.iloc[j]['x_min'], df.iloc[j]['y_min'], df.iloc[j]['x_max'], df.iloc[j]['y_max']])>=0.4:
                if (df.iloc[i]['class_id']==df.iloc[j]['class_id']):
                    try:
                        ds_test.drop(df.index[j], inplace=True)
                    except:
                        pass
            j+=1
        i+=1
# df.iloc[0]

In [17]:
def intersect(boxa, boxb):
    minx = max(boxa[0], boxb[0])
    miny = max(boxa[1], boxb[1])
    maxx = min(boxa[2], boxb[2])
    maxy = min(boxa[3], boxb[3])
    return [minx, miny, maxx, maxy]
def box_area(box):
    return (box[2]-box[0])*(box[3]-box[1])
def iou(box_a, box_b):
    box_i = intersect(box_a, box_b)
    box_i_area = box_area(box_i)
    box_a_area = box_area(box_a)
    box_b_area = box_area(box_b)
    u_area = box_a_area+box_b_area-box_i_area
    return (box_i_area/u_area, box_i_area, u_area)

In [18]:
def score_ap(model,dataloader,threshold=0.7):
    model.eval()
    tp_list = []
    target_labels_list = []
    scores_list = []
    labels_list = []
    iou_list = []
    size = 0
    print_threshold = 200
    with torch.no_grad():        
        for index, (imgs,targets) in enumerate(dataloader):
            imgs = list(img.to(device) for img in imgs)
            predictions = model(imgs)
            size += index

            for i, prediction in enumerate(predictions):
                
                img = imgs[i]
                target = targets[i]
                boxes = []
                scores = []
                labels = []
                for i,p in enumerate(prediction["scores"]):
                    
                    if p>threshold:
                        
                        boxes.append(prediction["boxes"][i].cpu().numpy())
                        scores.append(prediction["scores"][i].cpu().item())
                        labels.append(prediction["labels"][i].cpu().item())
                labels_np = np.array(labels)
                scores_np = np.array(scores)
                
                tp, labels, scores, iou = compute_tp(target["boxes"].cpu().numpy(), 
                                target["labels"].cpu().numpy(),
                                boxes, labels, labels_np, scores_np, iou_threshold=0.5)
                scores_list.extend(scores)
                labels_list.extend(labels)
    
                target_labels_list.extend(target["labels"])
                tp_list.extend(tp)
                iou_list.extend(iou)
        
        p, r, ap, f1, unique_labels =  ap_per_class(
                                        np.asarray(tp_list), 
                                        np.asarray(scores_list),
                                        np.asarray(labels_list), 
                                        np.asarray(target_labels_list))
    m_ap = np.mean(ap)
    return m_ap

In [19]:
def ap_per_class(tp, conf, pred_cls, target_cls):
    """ Compute the average precision, given the recall and precision curves.
    Source: https://github.com/rafaelpadilla/Object-Detection-Metrics.
    # Arguments
        tp:    True positives (list).
        conf:  Objectness value from 0-1 (list).
        pred_cls: Predicted object classes (list).
        target_cls: True object classes (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """

    # Sort by objectness
    i = np.argsort(-conf)
    tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]

    # Find unique classes
    unique_classes = np.unique(target_cls)

    # Create Precision-Recall curve and compute AP for each class
    ap, p, r = [], [], []
    for c in tqdm.tqdm(unique_classes, desc="Computing AP"):
        i = pred_cls == c
        n_gt = (target_cls == c).sum()  # Number of ground truth objects
        n_p = i.sum()  # Number of predicted objects

        if n_p == 0 and n_gt == 0:
            continue
        elif n_p == 0 or n_gt == 0:
            ap.append(0)
            r.append(0)
            p.append(0)
        else:
            # Accumulate FPs and TPs
            fpc = (1 - tp[i]).cumsum()
            tpc = (tp[i]).cumsum()

            # Recall
            recall_curve = tpc / (n_gt + 1e-16)
            r.append(recall_curve[-1])

            # Precision
            precision_curve = tpc / (tpc + fpc)
            p.append(precision_curve[-1])

            # AP from recall-precision curve
            ap.append(compute_ap(recall_curve, precision_curve))

    # Compute F1 score (harmonic mean of precision and recall)
    p, r, ap = np.array(p), np.array(r), np.array(ap)
    f1 = 2 * p * r / (p + r + 1e-16)

    return p, r, ap, f1, unique_classes.astype("int32")

def compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves.
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.0], recall, [1.0]))
    mpre = np.concatenate(([0.0], precision, [0.0]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

In [20]:
def compute_tp(target_boxes, target_labels, pred_boxes, pred_labels, pred_cls, conf, iou_threshold=0.5, print_fp=False):
    '''
    calculate true positives, align pred labels and conf scores with true positives indexes, IoUs of bounding boxes

    Args:
    target_boxes: REQUIRED, array of target boxes
    target_labels: REQUIRED, array of target labels
    pred_boxes: REQUIRED, array of prediction boxes
    pred_labels: REQUIRED, array of prediction labels
    pred_cls: REQUIRED, array of predicted object classes
    conf: REQUIRED, array of objectness value ranging from 0-1
    iou_threshold: OPTIONAL, float, threshold of min iou, default - iou_threshold=0.5

    Return value: array of true positives, new_pred_cls, new_conf, iou_return_arr 
    '''
    avg_iou = 0

    # all three will be set to length of target_labels
    true_positives = np.zeros(len(target_labels))
    new_pred_cls = np.full(len(target_labels), -1)
    new_conf = np.zeros(len(target_labels))

    false_positives = 0
    fp_array = []
    detected_boxes = []
    iou_return_arr = np.zeros(len(target_labels))
    

    for pred_i, pred_label in enumerate(pred_labels):
        
        if pred_label not in target_labels:
            fp_array.append(pred_label)
            false_positives += 1 
            continue
        
        pred_box = np.array([pred_boxes[pred_i][0], 
                             pred_boxes[pred_i][1], 
                             pred_boxes[pred_i][2], 
                             pred_boxes[pred_i][3]])
        iou_list=bbox_iou(np.expand_dims(pred_box,0), target_boxes) # calculate iou between prediction and target box
        
        box_index = np.argmax(iou_list)
        iou = iou_list[box_index]
        if iou >= iou_threshold and box_index not in detected_boxes and pred_i < len(true_positives):
            if pred_label != target_labels[box_index]:
                print(f"pred_label: {pred_label}, box_index,{box_index}, target_labels:{target_labels}, detected_boxes: {detected_boxes}")
            else:
                true_positives[pred_i] = 1
                new_pred_cls[pred_i] = pred_cls[pred_i]
                new_conf[pred_i] = conf[pred_i] 
                iou_return_arr[pred_i] = iou
                detected_boxes += [box_index]
        
    if (print_fp):
        print(fp_array)
    return true_positives, new_pred_cls, new_conf, iou_return_arr

In [21]:
def bbox_iou(box1, box2):
    """
    Returns the IoU of two bounding boxes
    """
    # Get the coordinates of bounding boxes
    b1_x1, b1_y1, b1_x2, b1_y2 = box1[:, 0], box1[:, 1], box1[:, 2], box1[:, 3]
    b2_x1, b2_y1, b2_x2, b2_y2 = box2[:, 0], box2[:, 1], box2[:, 2], box2[:, 3]
    
    # get the corrdinates of the intersection rectangle
    inter_rect_x1 = np.maximum(b1_x1, b2_x1)
    inter_rect_y1 = np.maximum(b1_y1, b2_y1)
    inter_rect_x2 = np.minimum(b1_x2, b2_x2)
    inter_rect_y2 = np.minimum(b1_y2, b2_y2)
    
    # Intersection area
    inter_area = np.clip(inter_rect_x2 - inter_rect_x1 + 1, 0, None) * np.clip(
        inter_rect_y2 - inter_rect_y1 + 1, 0, None
    )
    # Union Area
    b1_area = (b1_x2 - b1_x1 + 1) * (b1_y2 - b1_y1 + 1)
    b2_area = (b2_x2 - b2_x1 + 1) * (b2_y2 - b2_y1 + 1)
    iou = inter_area / (b1_area + b2_area - inter_area + 1e-16)
    return iou

In [22]:
def main ():
    model = torch.load("../input/private-model-save-dataset/model-save-5.pkl")
    model.eval()
    
    dataset_test = ChestXRayDataset('../input/vinbigdata-1024-image-dataset/vinbigdata', get_transform(), ds_test)
    
    data_loader_test = torch.utils.data.DataLoader(
        dataset_test, batch_size=1, shuffle=False, num_workers=4,
        collate_fn=utils.collate_fn)
    
    model.to(device)
    
    m_ap = score_ap(model, data_loader_test, threshold=0.9)
    print(m_ap)

In [23]:
main()

pred_label: 14, box_index,6, target_labels:[14  8  6  1 14  8  6 10 10  7  7], detected_boxes: [3]
pred_label: 12, box_index,0, target_labels:[11  8  7  7 11 11  5  5 12 14 12 14], detected_boxes: [3]
pred_label: 9, box_index,6, target_labels:[10  3  8 11  3  8  3 12 12  3  9  3 10 12  3 10  6  3  3  1 10], detected_boxes: []
pred_label: 14, box_index,2, target_labels:[ 4  1  3  9 14 10 12 12 11  3 14  3 14 11 14 12 12], detected_boxes: [10]
pred_label: 9, box_index,12, target_labels:[ 9  9  9 11  1  9  8 12  8  9  9  8  8  8  8  9  8  9  9  8 11  9  9  8
  9], detected_boxes: []
pred_label: 5, box_index,0, target_labels:[ 8 12  8 14  2  5  8 11  5  7 11 12  5  7], detected_boxes: [11]
pred_label: 9, box_index,2, target_labels:[ 3 11 10 12  9  3], detected_boxes: []
pred_label: 1, box_index,4, target_labels:[14  4 12  1 12], detected_boxes: []
pred_label: 9, box_index,1, target_labels:[ 1 10  9], detected_boxes: [0]
pred_label: 14, box_index,4, target_labels:[ 3 14  1 12 12 12 14], det

Computing AP: 100%|██████████| 15/15 [00:00<00:00, 397.17it/s]

0.13534595541085398
